### En este notebook se hará una limpieza superficial a los datos. Las tareas realizadas son:

-Correccion de nombres de columnas.

-Correccion de los caracteres especiales.

-Correcion de las casas cuyo Precio estaba en metros cuadrados.

-Division de la columna con los elementos divididos en una lista.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import re
pd.set_option('display.max_columns', None) #Para mostrar todas las columnas

In [2]:
df = pd.read_csv('casas_urbania.csv')
#df.columns = nuevas_cols #Corregimos el nombre con la lista de arriba.

In [3]:
nuevas_cols = ['Nro_casa','Adicionales', 'Ambientes', 'Antiguedad', 'Anunciante','Area_Terreno','Balneario','NroBanios',
               'Nro_pisos','Cocheras','Descripcion','Direccion','Disposicion','Dormitorios','EsCondominio',
              'Estado de Inmueble', 'Fecha_pub', 'Fila', 'Fondo','Frente', 'Generales', 'Luminosidad', 
               'Mantenimiento', 'Mascotas','Medidas','NoTieneAlgunDatoExtra','Pisos_construibles','TipoPuertaCochera',
               'Precio','Referencia','Servicios','Tipo','TipoCochera','Ubicacion','SinNombre','Urbaniz',
               'Uso_comercial','Uso_profesional','Area_constr','Area_total','Areas_comunes']

In [4]:
df.columns = nuevas_cols
df.head()

,Nro_casa,Adicionales,Ambientes,Antiguedad,Anunciante,Area_Terreno,Balneario,NroBanios,Nro_pisos,Cocheras,Descripcion,Direccion,Disposicion,Dormitorios,EsCondominio,Estado de Inmueble,Fecha_pub,Fila,Fondo,Frente,Generales,Luminosidad,Mantenimiento,Mascotas,Medidas,NoTieneAlgunDatoExtra,Pisos_construibles,TipoPuertaCochera,Precio,Referencia,Servicios,Tipo,TipoCochera,Ubicacion,SinNombre,Urbaniz,Uso_comercial,Uso_profesional,Area_constr,Area_total,Areas_comunes
0,0,NaN,NaN,NaN,ENACORP S.A.,NaN,NaN,NaN,NaN,NaN,ENACORP presenta Condominio Club Casas del Va...,Av. San Juan s/n - Alt. Km. 30 Pan. Norte - Ca...,NaN,NaN,Si,NaN,Publicado el 27.07.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"S/ 175,000",NaN,NaN,NaN,NaN,NaN,Casa0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"['Chimenea', 'DesagÃ¼e', 'Intercomunicador', '...","['BalcÃ³n', 'BaÃ±o de visitas', 'Cocina', 'Com...",38 aÃ±os,Rosa Chavez,179.20 m2,NaN,5.0,3.0,2.0,Linda Casa en San Juan de Lurigancho\r\r\r\n\...,"Calle Los Acebos S.j. Lurigancho, Lima, Lima",NaN,5 a mÃ¡s,NaN,Bueno,Publicado el 25.08.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,AutomÃ¡tico,"US$340,000",a 2 cdras de los prÃ³ceres de Independencia,"['Agua', 'Luz']",Casa,Paralelas,San Juan De Lurigancho-Lima,Casa1,NaN,NaN,NaN,349.37 m2,349.37 m2,NaN
2,2,"['DesagÃ¼e', 'Terma', 'Cocina con reposteros']","['Cocina', 'Comedor', 'Comedor diario', 'DepÃ³...",56 aÃ±os,Luz Elena Galindo Estrada,78.00 m2,NaN,3.0,3.0,NaN,"Chalet interno AT 77.70 m2, AC 178.65 m2. \r\...",Altura De Universidad San Ignacio De Loyola Ma...,NaN,5 a mÃ¡s,NaN,Bueno,Publicado el 25.08.19,NaN,NaN,NaN,"['Centros comerciales cercanos', 'Colegios cer...",Muy luminoso,NaN,Si,NaN,NaN,NaN,NaN,"US$156,000",NaN,"['Agua', 'Luz', 'Cable', 'Internet']",Casa,NaN,Magdalena Del Mar-Lima,Casa10,Orbea,NaN,NaN,178.00 m2,178.00 m2,NaN
3,3,"['DesagÃ¼e', 'Cocina con reposteros']","['BalcÃ³n', 'BaÃ±o de servicio', 'BaÃ±o de vis...",20 aÃ±os,Century 21 Capital,NaN,NaN,4.0,NaN,NaN,Venta de amplia Casa en estupenda zona de Mir...,Av. Diez Canseco Cerca A Parque Tradiciones// ...,NaN,5 a mÃ¡s,NaN,NaN,Publicado el 24.08.19,NaN,NaN,NaN,"['Centros comerciales cercanos', 'Cerca al mar...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"US$649,000",A poca caudra de Av. Ricardo Palma,"['Agua', 'Luz', 'Cable']",Casa,NaN,Miraflores-Lima,Casa100,NaN,NaN,NaN,215.00 m2,215.00 m2,['JardÃ\xadn']
4,4,['Jacuzzi'],"['BaÃ±o de servicio', 'BaÃ±o de visitas', 'Com...",NaN,No disponible,320.00 m2,NaN,5.0,3.0,2.0,"Linda casa, ubicada en la UrbanizaciÃ³n Los h...","Calle Los Himalayas Mz. O #03 La Molina, Los H...",Frente,5 a mÃ¡s,NaN,Excelente,Publicado el 04.07.19,NaN,NaN,12.0,"['Centros comerciales cercanos', 'Colegios cer...",Muy luminoso,NaN,NaN,NaN,NaN,NaN,Manual,"US$255,000",Casa Blanca dos pisos,"['Luz', 'TelÃ©fono', 'Cable', 'Internet', 'Gua...",Casa,Paralelas,La Molina-Lima,Casa1000,Los Huertos De La Molina,NaN,NaN,520.00 m2,320.00 m2,NaN


#### Haremos limpieza de los datos. Existen datos que están en un mal formato para hacer posteriormente el análisis.


In [5]:
df.drop(0,inplace= True) #Eliminamos la primera fila
df.drop('Nro_casa',axis=1,inplace=True)#Eliminamos la primera columna

In [6]:
df.Dormitorios = df.Dormitorios.replace({'5 a mÃ¡s':'5+'}) #Reemplazo

In [7]:
#Usamos expresiones regulares para corregir algunas columnas. 
df['Antiguedad'] = df['Antiguedad'].str.extract('(\d)', expand = True).astype('float32')

In [8]:
#Limpiaré las areas de los m2 con regex
df['Area_constr_m2'] = df['Area_constr'].str.extract('(\d{0,3}.?)')
df['Area_total_m2'] = df['Area_total'].str.extract('(\d{0,3}.?)')
df['Area_Terreno_m2'] = df['Area_Terreno'].str.extract('(\d{0,3}.?)')

In [9]:
dict_replacement = {"Ã³":"o","Ã±":"ni","Ã©":"e",
                    "Ã\xad":"i","Ã\x81":"A","Ã":"a","\\":"n","aº":"u","Ã¼":"u","¼":"u",
                    "Acceso para personas con discapacidad":"Acceso personas discapacidad",
                   "anx81rea ":"Area","\\r":" " ,"a¡":"a"}
for word, initial in dict_replacement.items():
    df['Servicios'] = df['Servicios'].str.replace(word, initial)
    df['Ambientes'] = df['Ambientes'].str.replace(word,initial)
    df['Areas_comunes'] = df['Areas_comunes'].str.replace(word,initial)
    df['Generales'] = df['Generales'].str.replace(word,initial)
    df['Adicionales'] = df['Adicionales'].str.replace(word,initial)
    df['Descripcion'] = df['Descripcion'].str.replace(word,initial)

In [10]:
#Hacer el reemplazo para solo una moneda
df['Precio']=df['Precio'].str.replace(",","")
index_casas_soles = df[(df['Precio'].str.startswith('US') == False) & (df['Precio'].str.startswith('Consultar') == False)].index
#Hasta acà los tengo almacenados en memoria.

In [11]:
df['Precio'].dtype

dtype('O')

In [12]:
df['Ubicacion'] = df['Ubicacion'].replace({'BreÃ±a-Lima':'Brenia-Lima', 'San Vicente De CaÃ±ete-Lima':'Caniete-Lima',
                                          'PariÃ±as-Piura':'Parianias-Piura','Los BaÃ±os Del Inca-Cajamarca':'Los Banios Del Inca-Cajamarca',
                                          'CaÃ±ete-Lima':'Caniete-Lima'})

In [13]:
#Limpio los US$ y S/ solo seleccionando los digits con regex
df['Precio'] = df['Precio'].str.extract('(\d{0,3},?\d{0,3},?\d{1,3}$)')

In [14]:
df['Ubicacion'] = df['Ubicacion'].str.replace(' ','')  #Para que no existan discordancias entre los strings
dist_prov = df['Ubicacion'].str.extractall(r'(\w*?)?-(\w*)')
dist_prov.columns = ['Distrito','Provincia']
dist_prov.reset_index(inplace = True)
dist_prov.drop(columns = ['match'],inplace=True)
dist_prov.set_index('level_0',inplace=True)

In [15]:
#Haré split a las columnas que tienen valores como listas.
ola1 = df['Servicios'].str.extractall('(\[\'\w*\s?\w*?\s?\w*?\'),?\]?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?')
ola2 = df['Ambientes'].str.extractall('(\[\'\w*\s?\w*?\s?\w*?\'),?\]?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?')
ola3 = df['Areas_comunes'].str.extractall('(\[\'\w*\s?\w*?\s?\w*?\'),?\]?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?')
ola4 = df['Generales'].str.extractall('(\[\'\w*\s?\w*?\s?\w*?\'),?\]?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?')
ola5 = df['Adicionales'].str.extractall('(\[\'\w*\s?\w*?\s?\w*?\'),?\]?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?,?\s?(\'\w*\s?\w*?\s?\w*?\')?\]?')

Ahora haremos join a las tablas que hemos creado

In [16]:
df['Precio'] = df['Precio'].astype('float64')
df.iloc[index_casas_soles - 1]['Precio'] = df.iloc[index_casas_soles -1]['Precio'].apply(lambda x: x*3.37)

C:\Users\pablo\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
df['Area_constr_m2'] = df['Area_constr_m2'].astype('float64') 
df['Area_total_m2'] = df['Area_total_m2'].astype('float64')
df['Area_Terreno_m2'] = df['Area_Terreno_m2'].astype('float64') 

plt.figure(figsize = (10,8))
ax = sns.scatterplot(x='Area_total_m2',y='Precio',data= df)

y_max = df['Precio'].max()
ticks = [0, 0.25*y_max, 0.5*y_max, 0.75*y_max, y_max]

ax.set_ylim(ax.get_ylim()) # set second y axis to have the same limits as the first y axis
ax.set_yticks(ticks) 
ax.set_yticklabels(ticks) # set the labels
ax.grid(True)

In [18]:
df.head()

,Adicionales,Ambientes,Antiguedad,Anunciante,Area_Terreno,Balneario,NroBanios,Nro_pisos,Cocheras,Descripcion,Direccion,Disposicion,Dormitorios,EsCondominio,Estado de Inmueble,Fecha_pub,Fila,Fondo,Frente,Generales,Luminosidad,Mantenimiento,Mascotas,Medidas,NoTieneAlgunDatoExtra,Pisos_construibles,TipoPuertaCochera,Precio,Referencia,Servicios,Tipo,TipoCochera,Ubicacion,SinNombre,Urbaniz,Uso_comercial,Uso_profesional,Area_constr,Area_total,Areas_comunes,Area_constr_m2,Area_total_m2,Area_Terreno_m2
1,"['Chimenea', 'Desagaue', 'Intercomunicador', '...","['Balcon', 'Banio de visitas', 'Cocina', 'Come...",3.0,Rosa Chavez,179.20 m2,NaN,5.0,3.0,2.0,Linda Casa en San Juan de Lurigancho \n \...,"Calle Los Acebos S.j. Lurigancho, Lima, Lima",NaN,5+,NaN,Bueno,Publicado el 25.08.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,AutomÃ¡tico,340000.0,a 2 cdras de los prÃ³ceres de Independencia,"['Agua', 'Luz']",Casa,Paralelas,SanJuanDeLurigancho-Lima,Casa1,NaN,NaN,NaN,349.37 m2,349.37 m2,NaN,349.0,349.0,179.0
2,"['Desagaue', 'Terma', 'Cocina con reposteros']","['Cocina', 'Comedor', 'Comedor diario', 'Depos...",5.0,Luz Elena Galindo Estrada,78.00 m2,NaN,3.0,3.0,NaN,"Chalet interno AT 77.70 m2, AC 178.65 m2. ...",Altura De Universidad San Ignacio De Loyola Ma...,NaN,5+,NaN,Bueno,Publicado el 25.08.19,NaN,NaN,NaN,"['Centros comerciales cercanos', 'Colegios cer...",Muy luminoso,NaN,Si,NaN,NaN,NaN,NaN,156000.0,NaN,"['Agua', 'Luz', 'Cable', 'Internet']",Casa,NaN,MagdalenaDelMar-Lima,Casa10,Orbea,NaN,NaN,178.00 m2,178.00 m2,NaN,178.0,178.0,78.0
3,"['Desagaue', 'Cocina con reposteros']","['Balcon', 'Banio de servicio', 'Banio de visi...",2.0,Century 21 Capital,NaN,NaN,4.0,NaN,NaN,Venta de amplia Casa en estupenda zona de Mir...,Av. Diez Canseco Cerca A Parque Tradiciones// ...,NaN,5+,NaN,NaN,Publicado el 24.08.19,NaN,NaN,NaN,"['Centros comerciales cercanos', 'Cerca al mar...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,649000.0,A poca caudra de Av. Ricardo Palma,"['Agua', 'Luz', 'Cable']",Casa,NaN,Miraflores-Lima,Casa100,NaN,NaN,NaN,215.00 m2,215.00 m2,['Jardanxadn'],215.0,215.0,NaN
4,['Jacuzzi'],"['Banio de servicio', 'Banio de visitas', 'Com...",NaN,No disponible,320.00 m2,NaN,5.0,3.0,2.0,"Linda casa, ubicada en la Urbanizacion Los hu...","Calle Los Himalayas Mz. O #03 La Molina, Los H...",Frente,5+,NaN,Excelente,Publicado el 04.07.19,NaN,NaN,12.0,"['Centros comerciales cercanos', 'Colegios cer...",Muy luminoso,NaN,NaN,NaN,NaN,NaN,Manual,255000.0,Casa Blanca dos pisos,"['Luz', 'Telefono', 'Cable', 'Internet', 'Guar...",Casa,Paralelas,LaMolina-Lima,Casa1000,Los Huertos De La Molina,NaN,NaN,520.00 m2,320.00 m2,NaN,520.0,320.0,320.0
5,NaN,"['Balcon', 'Banio de servicio', 'Banio de visi...",3.0,No disponible,150.00 m2,NaN,3.0,3.0,2.0,CASA DE TRES PLANTAS EN UNA DE LAS MEJORES ZO...,"Cuadra 9 Av.pardo Miraflores, Santa Cruz, Lima...",Frente,5+,NaN,Bueno,Publicado el 04.07.19,NaN,NaN,NaN,"['Centros comerciales cercanos', 'Cerca al mar...",Muy luminoso,NaN,Si,NaN,NaN,3.0,Manual,480000.0,ALTURA CUADRA 9 AV. PARDO,"['Agua', 'Luz', 'Telefono', 'Cable', 'Internet']",Casa,Separadas,Miraflores-Lima,Casa1001,Santa Cruz,No,Si,230.00 m2,150.00 m2,NaN,230.0,150.0,150.0


In [19]:
##Reviso outliers.
index_ = df[(df['Precio']<=10000)&(df['Ubicacion']=='Cusco-Cusco')].index #Los que tienes de ubicación Cusco
index_2 = df[(df['Precio']<=10000)].index #Los que tiene un precio que no es lógico

In [20]:
##Acá estoy corrigiendo los precios que no veo que sean lógicos...
a = df[df['Precio']<=10000]['Precio']*df[(df['Precio']<=10000)]['Area_total_m2'] #Corrijo ese precio (m2*Precio(m2))

df.iloc[index_2]['Precio'] = a #Reemplazo los precios que he corregido por sus valores en general.

C:\Users\pablo\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


Asumiré que las varibales más importantes son: Tipo_Casa, Dormitorios, Area_constr,Distrito, Departamento, Estado_inmueble,
                                                Ambientes.

Para ordenar mis dummies voy a tener que:

1.- Sacar una lista de valores únicos para cada nombre comenzando desde adelante.

2.- Iterar por sobre cada columna y cada elemento de la lista única, matcheando cuando los nombres son iguales.

3.- En cada iteración ubicar las columnas iguales y sumarlas entre sí

4.- Esta última suma, agregarla a un diccionario y posteriormente a un df

In [21]:
def merge_my_dummies(df_):
    df_cols = list(df_.columns) #Las columnas en lista
    data = pd.get_dummies(df_, columns = df_cols) #Hago dummies
    data_cols = list(data.columns)
    cols_limpias = [w.replace('[','') for w in data_cols] #Limpio los nombres
    lista_unica_elem = list(set(nombre_col.split('_')[1] for nombre_col in cols_limpias)) #Hago lista única
    dicc_inter = {}
    for nombre_col in lista_unica_elem:
        cols_to_sum = data.columns[data.columns.str.contains(nombre_col)] #para cada iteración agarra las cols que contienen lo q quiero
        series_to_join = data[cols_to_sum].iloc[:,:].sum(axis = 1) #Acà tengo la serie
        dicc_inter[nombre_col] = series_to_join #Lo agrego a un dict
    dataframe = pd.DataFrame.from_dict(dicc_inter) #Lo convierto a un dframe
    return dataframe

In [22]:
df_servicios = merge_my_dummies(ola1)
df_ambientes = merge_my_dummies(ola2)
df_arcomunes = merge_my_dummies(ola3)
df_generales = merge_my_dummies(ola4)
df_adicional = merge_my_dummies(ola5)

Ahora tendré que agregar esta data al df original

In [23]:
df_features = pd.concat([df_ambientes, df_servicios,df_arcomunes, df_generales, df_adicional], axis=1, sort=False)
df_features.fillna('NaN', inplace = True)
df_features.head()

,,'Comedor','anx81tico','Deposito','Banio de servicio','Banio independiente','Patio','Sala','Banio de visitas','Escritorio','Balcon','Comedor diario','Kitchenette','Terraza','Sotano','Jardanxadn Interno','Cocina','Sala de estar','Lavanderanxada','Closet','Walking Closet','Cuarto de servicio','Conexion a gas','Guardiananxada','Luz','Sistema de seguridad','Agua','Internet','Servicio de Limpieza','Telefono','Cable','Areade sauna','Club House','anx81reas verdes','Piscina','Guarderanxada','Parque interno','Areade BBQ','Gimnasio','Solarium','Hall de ingreso','Areadeportiva','Jardanxadn','Juegos para ninios','Sala de cine','Sala de internet','Cerca al mar','Ingreso independiente','Colegios cercanos','Frente al mar','Centros comerciales cercanos','Parques cercanos','Acceso personas discapacidad','Desagaue','Jacuzzi','Amoblado','Aire acondicionado','Chimenea','Calefaccion','Parrilla','Sauna','Cerco Electrico','Equipado','Portero electrico','Terma','Cocina con reposteros','Intercomunicador'
,match,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,0,0,0,0,0,1,1,1,1,0,1,0,0,1,1,1,0,0,1,0,0,1,0,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,1,0,0,0,1,0,1,0,1,1
2,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,1,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0
3,0,1,0,1,1,0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
5,0,1,0,0,1,0,1,1,1,0,1,1,0,1,0,0,1,1,1,1,0,1,0,0,1,0,1,1,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1,0,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df_features.reset_index(inplace = True)

In [25]:
#Vamos a hacer el cambio de las variables para poder hacer el respectivo join
df_features.set_index('level_0', inplace = True)
df_features.index.names = ['index']
df.index.names = ['index']

In [26]:
df_final = df.join(df_features,how='inner')
df_final = df_final.join(dist_prov,how='inner')
#df_final[df_final['Ubicacion'].isna()]
#Ahora borraremos las columnas de las que vienen nuestras dummies:
df_final.drop(['Servicios','Ambientes','Areas_comunes','Adicionales','Generales'],axis=1,inplace=True)

### Ahora que tenemos la data algo procesada, procederemos a limpiar los valores nulos en el siguiento notebook. La forma en la que quedó la data al final fue:

In [27]:
#Lo exporto en otro csv para poder limpiar los datos con más facilidad.
df_final.to_csv('data_pseudo_limpia2.csv')